In [1]:
import json
from StringIO import StringIO
import pandas as pd

In [2]:
import os
import sklearn as sk
from sklearn.linear_model import MultiTaskLasso, MultiTaskElasticNet, ElasticNet
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.metrics as skmet
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords, names
import datetime
from scipy.stats import ttest_ind 
import csv

In [3]:
listings = pd.read_csv('inside_airbnb/listings.csv')
list_summ = pd.read_csv('inside_airbnb/listings_summ.csv')
neighborhoods = pd.read_csv('inside_airbnb/neighbourhoods.csv')
reviews = pd.read_csv('inside_airbnb/reviews.csv')
reviews_summ = pd.read_csv('inside_airbnb/reviews_summ.csv')
calendar = pd.read_csv('inside_airbnb/calendar.csv')

In [4]:
def string_to_set(x):
    c = set()
    for w in x[1:-1].split(","):
        c.add(w)
        
    return c

def has_amenity(x, amen_):
    if amen_ in x:
        return 1
    pass

In [5]:
a = listings['amenities'].fillna('{}')

listings['amenities_set'] = a.map(string_to_set)
all_amenities = set()
    
for idx in listings['amenities_set'].index:
    all_amenities = all_amenities.union(listings['amenities_set'][idx])

In [10]:
b = set(['', '"translation missing: en.hosting_amenity_49"','"translation missing: en.hosting_amenity_50"'])

In [11]:
all_amenities = all_amenities.difference(b)

In [13]:
all_amenities

{'"24-Hour Check-in"',
 '"Air Conditioning"',
 '"Buzzer/Wireless Intercom"',
 '"Cable TV"',
 '"Carbon Monoxide Detector"',
 '"Elevator in Building"',
 '"Family/Kid Friendly"',
 '"Fire Extinguisher"',
 '"First Aid Kit"',
 '"Free Parking on Premises"',
 '"Hair Dryer"',
 '"Hot Tub"',
 '"Indoor Fireplace"',
 '"Laptop Friendly Workspace"',
 '"Lock on Bedroom Door"',
 '"Other pet(s)"',
 '"Pets Allowed"',
 '"Pets live on this property"',
 '"Safety Card"',
 '"Smoke Detector"',
 '"Smoking Allowed"',
 '"Suitable for Events"',
 '"Wheelchair Accessible"',
 '"Wireless Internet"',
 'Breakfast',
 'Cat(s)',
 'Dog(s)',
 'Doorman',
 'Dryer',
 'Essentials',
 'Gym',
 'Hangers',
 'Heating',
 'Internet',
 'Iron',
 'Kitchen',
 'Pool',
 'Shampoo',
 'TV',
 'Washer'}

In [14]:
for amen in all_amenities:
        
    if len(amen.split(' ')) == 1:
        listings['has_' + amen] = listings['amenities_set'].map(lambda x: has_amenity(x, amen))
        listings['has_' + amen] = listings['has_' + amen].fillna(0)
        continue
            
    if "" in amen:
        amen = amen[1:-1].replace(' ', '_')
            
    listings['has_' + amen] = listings['amenities_set'].map(lambda x: has_amenity(x, amen))
    listings['has_' + amen] = listings['has_' + amen].fillna(0)
    
has_amenties_list = []
for amen in all_amenities:
        
    if len(amen.split(' ')) == 1:
        has_amenties_list.append('has_' + amen)
        continue
            
    if "" in amen:
        amen = amen[1:-1].replace(' ', '_')
            
    has_amenties_list.append('has_' + amen)
    

In [22]:
has_amenties_list

['has_Other_pet(s)',
 'has_Essentials',
 'has_Carbon_Monoxide_Detector',
 'has_Elevator_in_Building',
 'has_Indoor_Fireplace',
 'has_Internet',
 'has_Washer',
 'has_Hangers',
 'has_Buzzer/Wireless_Intercom',
 'has_TV',
 'has_Gym',
 'has_Fire_Extinguisher',
 'has_Hot_Tub',
 'has_Cable_TV',
 'has_Air_Conditioning',
 'has_Suitable_for_Events',
 'has_Kitchen',
 'has_Family/Kid_Friendly',
 'has_Shampoo',
 'has_Heating',
 'has_Hair_Dryer',
 'has_Cat(s)',
 'has_Smoke_Detector',
 'has_Iron',
 'has_Free_Parking_on_Premises',
 'has_Pets_live_on_this_property',
 'has_Safety_Card',
 'has_Smoking_Allowed',
 'has_Dryer',
 'has_Pool',
 'has_Pets_Allowed',
 'has_Wheelchair_Accessible',
 'has_First_Aid_Kit',
 'has_Dog(s)',
 'has_Wireless_Internet',
 'has_Laptop_Friendly_Workspace',
 'has_Breakfast',
 'has_Doorman',
 'has_Lock_on_Bedroom_Door',
 'has_24-Hour_Check-in']

In [15]:
listings[has_amenties_list].isnull().sum()

has_Other_pet(s)                  0
has_Essentials                    0
has_Carbon_Monoxide_Detector      0
has_Elevator_in_Building          0
has_Indoor_Fireplace              0
has_Internet                      0
has_Washer                        0
has_Hangers                       0
has_Buzzer/Wireless_Intercom      0
has_TV                            0
has_Gym                           0
has_Fire_Extinguisher             0
has_Hot_Tub                       0
has_Cable_TV                      0
has_Air_Conditioning              0
has_Suitable_for_Events           0
has_Kitchen                       0
has_Family/Kid_Friendly           0
has_Shampoo                       0
has_Heating                       0
has_Hair_Dryer                    0
has_Cat(s)                        0
has_Smoke_Detector                0
has_Iron                          0
has_Free_Parking_on_Premises      0
has_Pets_live_on_this_property    0
has_Safety_Card                   0
has_Smoking_Allowed         

In [16]:
a = listings['host_verifications'].map(lambda x: x[1:-1]).map(lambda j: j.split(',')).map(lambda k: set(k))
all_host_verifications = set()
    
for w in a.index:
    all_host_verifications = all_host_verifications.union(a[w])
        
for w in all_host_verifications:
        
    if '' in w:
        w = w.strip()[1:-1].replace(' ', '_')
            
    listings['uses_' + w] = a.map(lambda x: has_amenity(x, w))
    listings['uses_' + w] = listings['uses_' + w].fillna(0)

In [19]:
uses_verification_list = []
for veri in all_host_verifications:
        
    if '' in veri:
        veri = veri.strip()[1:-1].replace(' ', '_')
    
    uses_verification_list.append('uses_' + veri)

In [21]:
listings[uses_verification_list].isnull().sum()

uses_phone             0
uses_linkedin          0
uses_email             0
uses_manual_online     0
uses_facebook          0
uses_amex              0
uses_phone             0
uses_sent_id           0
uses_jumio             0
uses_google            0
uses_weibo             0
uses_manual_offline    0
uses_kba               0
uses_kba               0
uses_reviews           0
dtype: int64

In [23]:
uses_verification_list

['uses_phone',
 'uses_linkedin',
 'uses_email',
 'uses_manual_online',
 'uses_facebook',
 'uses_amex',
 'uses_phone',
 'uses_sent_id',
 'uses_jumio',
 'uses_google',
 'uses_weibo',
 'uses_manual_offline',
 'uses_kba',
 'uses_kba',
 'uses_reviews']

In [25]:
listings['host_verifications'].map(lambda x: x[1:-1]).map(lambda j: j.split(',')).map(lambda k: set(k))

0               { 'phone',  'jumio',  'reviews', 'email'}
1       { 'phone',  'manual_online',  'jumio',  'revie...
2                                   { 'reviews', 'phone'}
3               { 'phone',  'jumio',  'reviews', 'email'}
4       { 'phone',  'kba',  'reviews',  'facebook', 'e...
5       { 'phone',  'kba',  'reviews',  'facebook', 'e...
6                 { 'phone',  'kba',  'reviews', 'email'}
7                 { 'phone',  'kba',  'reviews', 'email'}
8                 { 'phone',  'kba',  'reviews', 'email'}
9               { 'phone',  'jumio',  'reviews', 'email'}
10                        { 'phone',  'reviews', 'email'}
11                                    { 'phone', 'email'}
12                        { 'phone',  'reviews', 'email'}
13              { 'phone',  'jumio',  'reviews', 'email'}
14              { 'phone',  'jumio',  'reviews', 'email'}
15              { 'phone',  'jumio',  'reviews', 'email'}
16              { 'phone',  'jumio',  'reviews', 'email'}
17            

In [26]:
listings['amenities_set']

0       {Essentials, "Family/Kid Friendly", TV, Shampo...
1       {Essentials, "Carbon Monoxide Detector", Shamp...
2       {Essentials, "Family/Kid Friendly", TV, Dryer,...
3       {Hangers, Essentials, "Family/Kid Friendly", T...
4       {Essentials, "Indoor Fireplace", Internet, Was...
5       {Essentials, "Indoor Fireplace", Internet, Was...
6       {Essentials, TV, Heating, Dog(s), "Carbon Mono...
7       {TV, "Indoor Fireplace", Heating, Dog(s), "Wir...
8       {TV, "Indoor Fireplace", "Smoke Detector", Hea...
9       {Shampoo, "Laptop Friendly Workspace", Hangers...
10      {"translation missing: en.hosting_amenity_50",...
11      {"Laptop Friendly Workspace", Essentials, TV, ...
12      {Essentials, "Indoor Fireplace", Washer, Hange...
13      {"Laptop Friendly Workspace", Essentials, TV, ...
14      {Essentials, "Indoor Fireplace", Internet, Was...
15      {"Indoor Fireplace", Internet, Washer, Hangers...
16      {Essentials, "Indoor Fireplace", Washer, Hange...
17      {Essen